In [1]:
from tensorflow import keras
model = keras.models.load_model("sign_mnist_model.keras")

c:\Users\Abdel\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Abdel\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Abdel\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

In [2]:
import cv2
import numpy as np 
import os

In [3]:
MODEL_PATH = "sign_mnist_model.keras"
LABEL_MAP_PATH = "label_map.txt"
INVERT_FOR_GRAY = True
ROI_REL = (0.35, 0.15, 0.30, 0.60)

In [4]:

def load_label(path, num_classes):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            raw = [ln.strip() for ln in f if ln.strip()]
        labels = []
        for ln in raw:
            parts = [p.strip() for p in ln.split(",")]
            labels.append(parts[-1])
        return labels
    else:
        return [f"class_{i}" for i in range(num_classes)]


def preprocess(image, H, W, Convert):
    if Convert == 1:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        _, bw = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        bw = cv2.resize(bw, (W, H))
        return bw
    else:
        img = cv2.resize(image, (W, H))
        return img


In [5]:
import time

def main():
    # Charger le modèle
    model = keras.models.load_model(MODEL_PATH)
    H, W, C = model.input_shape[1:]   # ignorer batch_size

    num_classes = 24
    labels = load_label(LABEL_MAP_PATH, num_classes)

    cap = cv2.VideoCapture(0)

    fps_t0, fps_count = time.time(), 0

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        h, w = frame.shape[:2]

        # Définir ROI (Region of Interest)
        rx, ry, rw, rh = ROI_REL  # doit être défini avant
        x1, y1 = int(rx * w), int(ry * h)
        x2, y2 = int((rx + rw) * w), int((ry + rh) * h)

        # Dessiner rectangle ROI
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Extraire ROI
        inp_img = frame[y1:y2, x1:x2]

        # Prétraitement
        inp = preprocess(inp_img, H, W, Convert=1)

        # Adapter dimensions pour modèle
        inp = np.expand_dims(inp, axis=0)  

        # Prédiction
        pred = model.predict(inp, verbose=0)
        idx = int(np.argmax(pred))
        label = f"class : {labels[idx]}"

        fps_count += 1

        # Afficher résultat
        cv2.putText(frame, f"Pred: {label}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("webcam inference", frame)

        # Quitter avec 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()





In [ ]:
if __name__ == "__main__":
    main()